# ComputerVision - Real Time Project (Vehicle Counting)

In [1]:
# import basic libraries
import numpy as np
import pandas as pd
import cv2
from time import sleep # counting vehicle thats why i import the time

In [2]:
# Enabling the web camera
cap = cv2.VideoCapture('video.mp4') # video camera is enabling
# algo = cv2.createBackgroundSubtractorMOG2() # here so many noise is there compare to createBackgroundSubtractorKNN() 
algo = cv2.createBackgroundSubtractorKNN() # here less noise is there compare to createBackgroundSubtractorMOG2() 
# createBackgroundSubtractorMOG2 is one of the major Image Processing(like Image detection,image segmentation) task.

while True:
    ret,frame1 = cap.read()
    gray = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3),5)
    # Applaying preprocessing in each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5,5))) # dilate(reduce the noise value) means filling the gap in the detected object
                                                # fill 5 by 5 matrix
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5)) # kernel will create structure because to use the morphology operation(noise data get removed)
    dilatada = cv2.morphologyEx(dilat,cv2.MORPH_CLOSE,kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)
    counterShape = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.imshow("Detector Video",dilatada)

    if cv2.waitKey(1)==13:
        break
cap.release()
cv2.destroyAllWindows()


In [3]:
min_width_rect = 80
min_height_rect = 80
offset = 6
delay = 60
carros = 0 # count the number of vehicle detect
count_line_pos = 550
detec = []

def central_handle(x,y,w,h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x + x1
    cy = y + y1
    return cx, cy
    

# Enabling the web camera
cap = cv2.VideoCapture('video.mp4') # video camera is enabling
# algo = cv2.createBackgroundSubtractorMOG2() # here so many noise is there compare to createBackgroundSubtractorKNN() 
algo = cv2.createBackgroundSubtractorKNN() # here less noise is there compare to createBackgroundSubtractorMOG2() 
# createBackgroundSubtractorMOG2 is one of the major Image Processing(like Image detection,image segmentation) task.

while True:
    ret,frame1 = cap.read()
    gray = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3),5)
    # Applaying preprocessing in each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5,5))) # dilate(reduce the noise value) means filling the gap in the detected object
                                                # fill 5 by 5 matrix
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5)) # kernel will create structure because to use the morphology operation(noise data get removed)
    dilatada = cv2.morphologyEx(dilat,cv2.MORPH_CLOSE,kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)
    counterShape, h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.line(frame1, (25,count_line_pos), (1200,count_line_pos), (0,0,255),3) # 
    # start line = 25,count_line_pos
    # end line = 1200,count_line_pos
    # line color = (0,0,255)
    # thickness of the line = 3

    for (i,c) in enumerate(counterShape):
        (x,y,w,h) = cv2.boundingRect(c)
        validar_contorno = (w>=min_width_rect) and (h>=min_height_rect)
        if not validar_contorno:
            continue
        cv2.rectangle(frame1, (x,y), (x+w, y+h), (0,255,0), 2)
        # original image = frame1
        # x axis =  (x,y)
        # y - axis = (x+w, y+h)
        # color  = (0,255,0)
        # thickness = 2
        cv2.putText(frame1, "Vehicle" + str(carros), (x, y-20), cv2.FONT_HERSHEY_TRIPLEX, 1, (255,0,255),2)
        center = central_handle(x,y,w,h)
        detec.append(center)
        cv2.circle(frame1, center, 4, (200,100,50),-1)


        # Loop Function - checking if the vehicle crossed the line
        for (x,y) in detec:
            if y<(count_line_pos+offset) and y>(count_line_pos-offset):
                carros+=1
                cv2.line(frame1, (25,count_line_pos), (1200,count_line_pos),(0,0,255), 3)
                detec.remove((x,y))
                print("Car is detecting :" +str(carros))

    cv2.putText(frame1, "Vehicle" + str(carros), (x, y-20), cv2.FONT_HERSHEY_TRIPLEX, 1, (255,0,255),2)
        

    cv2.imshow("Original Video",frame1)

    if cv2.waitKey(1)==13:
        break
cap.release()
cv2.destroyAllWindows()


Car is detecting :1
Car is detecting :2
Car is detecting :3
Car is detecting :4
Car is detecting :5
Car is detecting :6
Car is detecting :7
Car is detecting :8
Car is detecting :9
Car is detecting :10


# Tomato_Leaf_Image

In [ ]:
# download tomato leaf dataset
!unzip '/content/drive/MyDrive/Tomato_Leaf/tomato_leaf_images.zip'

In [ ]:
from pathlib import Path
training_data = Path('tomato_leaf_images/train')
validation_data = Path('tomato_leaf_images/val')

In [ ]:
labels_dict = {'AmericanLeafMiner':0, 'Healthy':1,'MagnesiumDeficiency':2,
               'SerpentineLeafMiner':3}

In [ ]:
import os
import pandas as pd
train_df = []
for folder in os.listdir(training_data):
  imgs_path = training_data / folder
  imgs = sorted(imgs_path.glob('*.jpg'))
  for img_name in imgs:
    train_df.append((str(img_name),labels_dict[folder] ))
train_df = pd.DataFrame(train_df, columns = ['image', 'label'], index=None)

In [ ]:
train_df.shape

In [ ]:
train_df